In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from collections import Counter
import random
import re
import datasets
import tqdm
import math
from functools import partial
import math
import argparse
import os
import collections
import json
import sentencepiece
import shutil
import copy
import multiprocessing


torch.set_float32_matmul_precision("high")

c:\Users\ilsai\Downloads\a1 (1)\a1\a1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
## you can modify some options such as batch_size, depending on your environments  

embedding_training_args = {
    "batch_size": 32768,
    "epochs": 10,
    "lr": 1e-3,
    "device": "cpu",  # CUDA 호환성 문제로 CPU 사용
    "embedding_dim": 512,

    "vocab_size": 50000,
    "min_freq": 5,

    "gradient_accumulate_steps": 1,
}

tokenizer_args = {
    "vocab_size": 50000,
    "min_freq": 5,
}

### Tokenizer from Corpus (WhiteSpace and BPE)

In [ ]:
class Tokenizer:
    def __init__(self, 
                 max_vocab_size=10000, 
                 special_tokens=[],
                 pad_token="<|PAD|>",
                 unk_token="<|UNK|>",
                 bos_token="<|BOS|>",
                 eos_token="<|EOS|>"):
        self.max_vocab_size = max_vocab_size
        self.tokenizer = None

        self.pad_token = pad_token
        self.pad_token_id = 0
        self.unk_token = unk_token
        self.unk_token_id = 1
        self.bos_token = bos_token
        self.bos_token_id = 2
        self.eos_token = eos_token
        self.eos_token_id = 3

        self.special_tokens = [self.pad_token, self.unk_token, self.bos_token, self.eos_token] + special_tokens
        self.additional_special_tokens = special_tokens
    
    def save(self, path):
        os.makedirs(path, exist_ok=True)
        with open(os.path.join(path,"mics.json"), "w") as f:
            json.dump({
                "max_vocab_size": self.max_vocab_size,
                "special_tokens": self.special_tokens,
                "additional_special_tokens": self.additional_special_tokens,
            }, f)
    
    def load(self, path):
        with open(os.path.join(path,"mics.json"), "r") as f:
            mics = json.load(f)
            self.max_vocab_size = mics["max_vocab_size"]
            self.special_tokens = mics["special_tokens"]
            self.additional_special_tokens = mics["additional_special_tokens"]

class WhitespaceTokenizer(Tokenizer):
    def __init__(self, min_count=5, max_vocab_size=10000, special_tokens=[]):
        super().__init__(max_vocab_size, special_tokens)
        self.min_count = min_count
        self.vocab = {}

        assert len(special_tokens) == len(set(special_tokens)), "Duplicate special tokens are not allowed."
        assert len(special_tokens) < max_vocab_size, "Special tokens exceed max vocab size."

    def __len__(self):
        return len(self.vocab)

    def normalize(self, text):
        return re.sub(r'[^\w\s]', '', text).lower()

    def fit(self, dataset):
        token_counter = Counter()
        for text in tqdm.tqdm(dataset, desc="WhitespaceTokenizer fitting..."):
            tokens = self.normalize(text).split()
            token_counter.update(tokens)
        
        vocab = [(word,count) for word, count in token_counter.items() if count >= self.min_count]
        vocab = sorted(vocab, key=lambda x: -x[1])
        vocab = vocab[:(self.max_vocab_size - len(self.special_tokens))]
        vocab = [word for word, _ in vocab]
        vocab = self.special_tokens + vocab
        
        self.sample_weights = []
        for i, word in enumerate(vocab):
            self.vocab[word] = i
            self.sample_weights.append(math.log(token_counter[word]) if word in token_counter else -987654321)

    def tokenize(self, text, add_special_tokens=False):
        tokens = self.normalize(text).split()
        if add_special_tokens:
            tokens = ["<|BOS|>"] + tokens + ["<|EOS|>"]
        return [self.vocab.get(token, self.vocab["<|UNK|>"]) for token in tokens]

    def save(self, path):
        super().save(path)
        with open(os.path.join(path,"word2idx.json"), "w") as f:
            json.dump(self.vocab, f)
        with open(os.path.join(path,"sample_weights.json"), "w") as f:
            json.dump(self.sample_weights, f)
        
    
    def load(self, path):
        super().load(path)
        with open(os.path.join(path,"word2idx.json"), "r") as f:
            self.vocab = json.load(f)
        with open(os.path.join(path,"sample_weights.json"), "r") as f:
            self.sample_weights = json.load(f)

class BPETokenizer(Tokenizer):
    def __init__(self, max_vocab_size=10000, special_tokens=[]):
        super().__init__(max_vocab_size, special_tokens)
        self.bpe = None
        self.sample_weights = None
        assert len(special_tokens) == len(set(special_tokens)), "Duplicate special tokens are not allowed."
        assert len(special_tokens) < max_vocab_size, "Special tokens exceed max vocab size."
    
    def __len__(self):
        return self.bpe.get_piece_size()

    def fit(self,dataset, save_path):
        print("Training BPE Tokenizer...")
        os.makedirs(save_path, exist_ok=True)
        prefix = os.path.join(save_path,"bpe")
        sentencepiece.SentencePieceTrainer.train(
            sentence_iterator=iter(dataset),
            model_prefix=prefix,
            vocab_size=self.max_vocab_size,
            max_sentence_length=100000,
            shuffle_input_sentence=False,
            byte_fallback=True,
            num_threads=32,
            pad_id=0,
            pad_piece="<|PAD|>",
            unk_id=1,
            unk_piece="<|UNK|>",
            bos_id=2,
            bos_piece="<|BOS|>",
            eos_id=3,
            eos_piece="<|EOS|>",
            user_defined_symbols=self.additional_special_tokens
        )
        self.bpe = sentencepiece.SentencePieceProcessor()
        self.bpe.load(prefix + ".model")
        with open(prefix+".vocab", "r", encoding="utf-8") as f:
            self.vocab = {}
            self.sample_weights = []
            for l in f:
                token, weight = l.strip().split("\t")
                self.vocab[token] = len(self.vocab)
                self.sample_weights.append(weight if token not in self.special_tokens else -987654321)

    def tokenize(self, text, add_special_tokens=False):
        tokens = self.bpe.encode(text, out_type=int)
        if add_special_tokens:
            tokens = [self.bpe.bos_id] + tokens + [self.bpe.eos_id]
        return tokens

    def save(self, path):
        super().save(path)
    
    def load(self, path):
        super().load(path)
        self.bpe = sentencepiece.SentencePieceProcessor()
        self.bpe.load(os.path.join(path,"bpe.model"))
        with open(os.path.join(path,"bpe.vocab"), "r", encoding="utf-8") as f:
            self.vocab = {}
            self.sample_weights = []
            for l in f:
                token, weight = l.strip().split("\t")
                self.vocab[token] = len(self.vocab)
                self.sample_weights.append(weight)

# Building Vocab and Tokenizer from Corpus  

In [10]:

dataset = datasets.load_dataset("abisee/cnn_dailymail",'3.0.0')
corpus_train_dataset = dataset['train'].select(range(50000))
corpus_vaildation_dataset = dataset['validation']

In [13]:
if os.path.exists("output/whitespace_tokenizer"):
    white_space_tokenizer = WhitespaceTokenizer()
    white_space_tokenizer.load("output/whitespace_tokenizer")
else:
    white_space_tokenizer = WhitespaceTokenizer(tokenizer_args['min_freq'],tokenizer_args['vocab_size'])
    white_space_tokenizer.fit(corpus_train_dataset['article'])
    white_space_tokenizer.save("output/whitespace_tokenizer")

if os.path.exists("output/bpe_tokenizer"):
    bpe_tokenizer = BPETokenizer()
    bpe_tokenizer.load("output/bpe_tokenizer")
else:
    bpe_tokenizer = BPETokenizer(tokenizer_args['vocab_size'])
    bpe_tokenizer.fit(corpus_train_dataset['article'], "output/bpe_tokenizer")
    bpe_tokenizer.save("output/bpe_tokenizer")

Training BPE Tokenizer...


### Loading IMDB Dataset, Data Precessing Functions   

In [14]:

from sklearn.metrics import accuracy_score

## Loading IMDB dataset 
imdb_dataset = datasets.load_dataset("stanfordnlp/imdb")


def nb_collate_fn(examples, tokenizer):
    tokens = [tokenizer.tokenize(e['text']) for e in examples]
    labels = [e['label'] for e in examples]

    max_len = max(len(t) for t in tokens)
    input_ids = [t + [tokenizer.pad_token_id] * (max_len - len(t)) for t in tokens]
    attention_mask = [[1] * len(t) + [0] * (max_len - len(t)) for t in tokens]
    return {
        "input_ids": torch.LongTensor(input_ids),
        "attention_mask": torch.LongTensor(attention_mask),
        "labels": torch.LongTensor(labels),
    }


c:\Users\ilsai\Downloads\a1 (1)\a1\a1\Lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ilsai\.cache\huggingface\hub\datasets--stanfordnlp--imdb. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 678791.02 examples/s]


## Module for Naive Bayesian Models (nn.Module) -- Your codes are required 

In [ ]:


'''
nn.Module
a. 파라미터 딕셔너리 생성 (학습 가능한 파라미터 저장용)
b. 학습 안되는 텐서 (buffer) 생성
c. 모듈 딕셔너리 (하위 모듈 저장용)
d. training flag 설정 (True & False)

class Module:
    def __init__(self):
        self._parameters = {}  # Parameter 객체 저장
        self._buffers = {}     # Buffer 텐서 저장
        self._modules = {}     # 하위 모듈 저장
        self.training = True
'''
class DenseMultinomialNaiveBayes(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes, labels):
        super(DenseMultinomialNaiveBayes, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.class_context = nn.Parameter(torch.randn(embedding_dim, num_classes)) #class context vector 을 파라미터화 해서 랜덤으로 초기화


        label_counter = Counter(labels)
        self.register_buffer("class_count", torch.zeros(num_classes)) #log class prior 계산 위해서
        for c, count in label_counter.items():
            self.class_count[c] = count
        self.register_buffer("log_class_prior",torch.log(self.class_count / len(labels) + 1e-9)) #log class prior 은 학습 안시킴 그냥 특정 값이 어느 클래스 들어갈지 예측만 시킴
        # self.log_class_prior = nn.Parameter()
        self.num_classes = num_classes
        self.vocab_size = vocab_size
    '''
    - vᵢ: 단어 i의 embedding vector [embedding_dim]
    - cₖ: 클래스 k의 context vector [embedding_dim]
    torch.matmul 만 자동으로 broadcasting 됨 -> 나머지 
    elementwise 곱 = 이미 있는 정보에 마스크나 가중치 씌울때: attention mask, sigmoid gate, dropout, loss weighting
    matmul - 정보를 새 공간으로 투사 (projection): linear layer, embedding lookup, attention score, classifier
    '''
    def forward(self,input_ids,attention_mask,labels=None):
        word_embeds = self.embeddings(input_ids) #[batch_size, seq_len, embedding_dim]
        logits = torch.matmul(word_embeds, self.class_context) #softmax 전 내적값을 취함
        log_probs = F.log_softmax(logits, dim=-1) #dim -1 은 마지막 차원인 특정 값이 어떤 클래스에 존재할까에 대한 내용임
        mask = attention_mask.unsqueeze(-1).float() #마지막에 차원 1개 추가해주는게 unsqueeze
        masked_log_probs = (log_probs*mask).sum(dim=1) #앞부분
        log_joint = masked_log_probs + self.log_class_prior #뒷부분 

        if labels is not None:
            loss = -log_joint[range(log_joint.size(0)), labels].mean()#배치 내 각 샘플 loss 의 평균 반환
            return loss, log_joint

        return log_joint

        ### YOUR CODES END ###

ft_training_args = copy.deepcopy(embedding_training_args) #원본 복사해서 원본은 건드리지 않는 선에서 수정 가능하게 만든 상태
ft_training_args['batch_size'] = 32
ft_training_args['epochs'] = 10
ft_training_args['lr']=1e-4
ft_training_args['hidden_dim'] = 512
ft_training_args['device'] = 'cpu'  # CUDA 호환성 문제로 CPU 강제 사용

### Model Training with Validation and Checkpointing

In [ ]:
def train(model, train_dataset, valid_dataset, collate_fn, train_args, prefix):
    optimzier = optim.Adam(model.parameters(), lr=train_args["lr"]) #옵티마이저 설정

    train_dataloader = DataLoader(train_dataset, batch_size=train_args['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=0) #shuffle = true 는 매 에폭마다 섞기 collate_fn 은 배치 만들기
    valid_dataloader = DataLoader(valid_dataset, batch_size=train_args['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=0) #train_args 는 결국 ft_training_args 를 참조

    total_steps = len(train_dataloader) * train_args['epochs']

    best_loss = 987654321
    
    output_path = os.path.join("output", prefix)
    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, "train_args.json"), "w") as f:
        json.dump(train_args, f)

    pbar = tqdm.tqdm(total=total_steps, desc="training")
    for epoch in range(train_args['epochs']):
        pbar.set_description(f"Epoch {epoch+1}/{train_args['epochs']}")
        move_avg_loss = []
        model.train()
        for i, batch in enumerate(train_dataloader):
            batch = {k:v.to(train_args['device']) if isinstance(v,torch.Tensor) else v for k,v in batch.items()}

            loss = model(**batch)
            if isinstance(loss, tuple):  # (loss, logits) 튜플인 경우 loss만 추출
                loss = loss[0]
            loss = loss / train_args['gradient_accumulate_steps']
            if loss.size() != torch.Size([]):
                loss = loss.mean()
            loss.backward()
            
            if (i+1) % train_args['gradient_accumulate_steps'] == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), 3.0)
                optimzier.step()
                optimzier.zero_grad()

            move_avg_loss.append(loss.item()) 
            if len(move_avg_loss) > 100: move_avg_loss.pop(0)
            pbar.set_postfix_str(f"loss: {sum(move_avg_loss)/len(move_avg_loss):.04f} lr: {optimzier.param_groups[0]['lr']:.2e}")
            pbar.update(1)
        
        model.eval()
        with torch.no_grad():
            eval_loss = 0
            for i, batch in enumerate(valid_dataloader):
                with torch.no_grad():
                    batch = {k:v.to(train_args['device']) if isinstance(v,torch.Tensor) else v for k,v in batch.items()}
                    loss = model(**batch)
                    if isinstance(loss, tuple):  # (loss, logits) 튜플인 경우 loss만 추출
                        loss = loss[0]
                    if loss.size() != torch.Size([]):
                        loss = loss.mean()
                    eval_loss += loss.item()
                    pbar.set_postfix_str(f"val_loss: {eval_loss / (i+1):.04f}")
        eval_loss /= len(valid_dataloader)
        print(f"\n[Epoch {epoch+1}/{train_args['epochs']}] Validation Loss: {eval_loss:.04f}")

        if eval_loss < best_loss:
            best_loss = eval_loss
            
            torch.save(model.state_dict(), os.path.join(output_path,"best_model.pth"))
            print(f"Model Saved! best loss: {best_loss:.04f}")

    pbar.close()

## Call Model Training for Finetuning -- Your codes are required  

In [46]:
def nb_finetuning(model_class, tokenizer, train_dataset, valid_dataset, training_args, prefix, embedding=None):
    ## YOUR CODE (single line)
    model = model_class(
    len(tokenizer),  # vocab_size
    training_args['hidden_dim'],  # embedding_dim
    len(set([e['label'] for e in train_dataset])),  # num_classes (동적) 즉 분류 들어가야 하는 클래스 개수
    [e['label'] for e in train_dataset])  # labels 즉 실제 학습 데이터의 모든 레이블 리스트

    model = model.to(training_args['device'])
    if embedding is not None:
        model.embeddings.weight.data.copy_(embedding)
    # model = torch.compile(model)  # Windows에서 torch.compile 호환성 문제로 비활성화
    train(model, train_dataset, valid_dataset, lambda x:nb_collate_fn(x,tokenizer), training_args, prefix)
    best_model_statedict = torch.load(os.path.join("output",prefix,"best_model.pth"))
    model.load_state_dict(best_model_statedict)
    return model

def nb_evaluate(model, dataset, tokenizer):
    model.eval()
    with torch.no_grad():
        dataloader = DataLoader(dataset, batch_size=32, shuffle=False, collate_fn=partial(nb_collate_fn, tokenizer=tokenizer))
        predicted = []
        for batch in tqdm.tqdm(dataloader):
            input_ids = batch['input_ids'].to(embedding_training_args['device'])
            attention_mask = batch['attention_mask'].to(embedding_training_args['device'])
            logits = model(input_ids, attention_mask)
            predicted.extend(logits.argmax(dim=1).tolist())
    accuracy = accuracy_score([e['label'] for e in dataset], predicted)
    print(f"Accuracy: {accuracy:.04f}")


### Finetuning Naive Bayes Model based on Word Embedding 

In [47]:
random_embedding = torch.nn.Embedding(tokenizer_args['vocab_size'], embedding_training_args['embedding_dim']).weight.data

nb_wt_random = nb_finetuning(DenseMultinomialNaiveBayes, white_space_tokenizer, imdb_dataset['train'], imdb_dataset['test'], ft_training_args, "nb_wt_random", random_embedding)

nb_evaluate(nb_wt_random, imdb_dataset['test'], white_space_tokenizer)


[Epoch 1/10] Validation Loss: 633.3262
Model Saved! best loss: 633.3262



[Epoch 2/10] Validation Loss: 345.9706
Model Saved! best loss: 345.9706



[Epoch 3/10] Validation Loss: 280.1309
Model Saved! best loss: 280.1309



[Epoch 4/10] Validation Loss: 246.2858
Model Saved! best loss: 246.2858



[Epoch 5/10] Validation Loss: 225.5778
Model Saved! best loss: 225.5778


...


[Epoch 8/10] Validation Loss: 195.0167
Model Saved! best loss: 195.0167



[Epoch 9/10] Validation Loss: 189.7700
Model Saved! best loss: 189.7700



[Epoch 10/10] Validation Loss: 185.6721
Model Saved! best loss: 185.6721






























































































































































100%|██████████| 782/782 [00:05<00:00, 141.69it/s]


Accuracy: 0.6731
